In [1]:
import os
import sys
import json
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))

# Setup stuff.  The cell just supports the workbook, you can ignore it

EXAMPLE_DO_FOLDER = os.path.join(os.getcwd(), "example_do_folder")
INSERTION_FOLDER = os.path.join(EXAMPLE_DO_FOLDER, "hello/do_examples")

# def show_loadable(at):
#     with open(os.path.join(INSERTION_FOLDER, at)) as file:
#         contents = file.read()
#     with_bar = "\n  | ".join(contents.split("\n"))
#     print(F"\n### SHOWING MODULE {'.../'+at!r}\n  | {with_bar}\n\n")

def show(base):
    with open(do.get_base(base).__file__) as file:
        contents = file.read()
    with_bar = "\n  | ".join(contents.split("\n"))
    print(F"\n### SHOWING MODULE {'.../'+base+'.py'!r}\n  | {with_bar}\n\n")

def run_do(*args, **kwargs):
    parts = [repr(x) for x in args] + [F"{k}={v!r}" for k, v in kwargs.items()]
    print(F"do({', '.join(parts)})")
    result = do(*args, **kwargs)
    print(F"--> {result!r}")
    print()


print(F"### INSERTION FOLDER        = {INSERTION_FOLDER!r}")
print(F"### DO FOLDER (in Jupyter)  = {EXAMPLE_DO_FOLDER}")
sys.path.append(os.path.dirname(os.path.dirname(EXAMPLE_DO_FOLDER)))
from dvc_dat import do   # Add all loadables BEFORE loading this module
# do.set_do_folder(dat_config.do_folder)
print(F"## DO FOLDER (in do module) = {do.do_folder!r}")
if EXAMPLE_DO_FOLDER != do.do_folder:
    print(f"WARNING: EXAMPLE_DO_FOLDER used by jupyter = {EXAMPLE_DO_FOLDER} does not match {do.do_folder}")
if not os.path.exists(INSERTION_FOLDER):
    input(f"WARNING: INSERTION_FOLDER {INSERTION_FOLDER!r} not found.")
    os.makedirs(INSERTION_FOLDER)
print("\n\n\n\n")


# Example Data

### Imports

In [2]:
from dvc_dat import dat_tools as dt
from dvc_dat import Dat, do

### Some Example Dats
These Dat and point_fns are used to build and show DataFrames.


In [3]:
TMP_PATH1 = "/tmp/job_test1"
spec1 = {"main": {"my_key1": "my_val1", "my_key2": "my_val2"}}
dat1 = Dat.create(spec=spec1, path=TMP_PATH1)

TMP_PATH2 = "/tmp/job_test2"
spec2 = {"main": {"my_key1": "my_val1", "my_key2": "my_val2"}, "other": "key_value"}
dat2 = Dat.create(spec=spec2, path=TMP_PATH2)


.

.

## Creating DataFrames from Dats and point_fns

#### Zero point_fns applied to zero Dats yields an empty DataFrame

In [4]:
dt.from_dat([], [])  # Empty lists return empty DataFrames

#### Two metrics applied to two dats yields a DataFrame w/ two rows & 2 cols

In [5]:
def always_17(_dat):
    return 17


def always_18(_dat):
    return 18

dt.from_dat([dat1, dat2], [always_17, always_18])

#### Metric functions can also be expressed as string and loaded using 'do'

In [6]:
do.register_module("registered_cube",
                           "example_do_folder.dat_tools_examples.cube_hello",
                   allow_redefine=True)
dt.from_dat([dat1], ["registered_cube.always_5"])

#### Multi-valued Metric Functions
Metric functions (also called point_fns) can return a dict of metric values instead of just returning a single value.

In [7]:
fns = [always_17, lambda dat: {"val1": 111, "val2": 2222}]
dt.from_dat([dat1, dat2], point_fns=fns)

#### Multi-POINT Metric Functions
Finally, a point_fn can return a LIST of dicts, each dict represents a row in the DataFrame.  In all cases, all point_fns are applied to all dats, and the return values are appended to the DataFrame.  These different kinds of point functions can be combined, so scalar values and dict values are all merged into a single row for each dat, and lists of dicts are expanded into multiple rows.

In [8]:
point_fns = [always_17, always_18, lambda dat: [{"val1": 1, "val2": 2}, {"val1": 11}]]
dt.from_dat([dat1], point_fns)

Above, you can see that the results of the always_17 and always_18 functions were merged into a single row as before, and the lambda returned to more data points expressed as two additional rows in the DataFrame.

# OUTPUT TO EXCEL
The 'to_excel' function writes the DataFrame to an Excel file.
To test this, we use the points_fn below 'load_points_json' it reads a list of data points from each dat by looking in the dat folder for a file named 'points.json'.  This is used to construct a large DataFrame from three Dats in the "Retail Data" DatContainer.

In [9]:
def load_points_json(dat: Dat):
    with open(f"{dat.get_path()}/points.json") as f:
        return json.load(f)


In [10]:
df = dt.from_dat("Datasets/Retail Data", [load_points_json])
df

In [11]:
dt.to_excel(df, show=True)

#### The 'sheets' parameter
The 'sheets' to_excel parameter can be used to split a data frame into separate Excel sheets according to the value of the indicated column.  Here we split the DataFrame so each city is on its own sheet.

In [12]:
dt.to_excel(df, sheets=["list"], show=True)

Providing multiple columns in the 'sheets' parameter will create a sheets for the cross-product of column values as shown here:

In [13]:
dt.to_excel(df, sheets=["list", "Store"], show=True)

#### The 'docs' parameter
Another way to slice a DataFrame is into separate Excel documents.  Here we split the DataFrame by month:

In [14]:
dt.to_excel(df, docs=["Month"], show=True)

#### The 'add_formatted_column' parameter
Often a report can be tidied up by combining related columns into single semantically
unified ones.   The 'add_formatted_columns' defines a simple formatting syntax based
on Python's format fn. Here we combine the 'Metric' and 'Value' columns into the new
'Result' column.

In [15]:
dt.to_excel(df, sheets=["Store", "Month"], show=True,
            formatted_columns=["Result <== {0} {1} <==  Value, Metric"])

#### The 'columns' parameter
The 'columns' parameter also can be used to trim the Excel sheet(s) to only include the listed
columns.  This can also be used to "tidy up" the output.

In [16]:
dt.to_excel(df, sheets=["Store", "Month"], show=True,
            columns=["list", "Product", "Result"],
            formatted_columns=["Result <== {0} {1} <==  Value, Metric"])

#### The 'transform' parameter
The formatted_columns parameter was included as combining columns is a common
special case.  More generally, one can use the 'transform' parameter to perform
arbitrary transformation of the dataframe before writing it to Excel.  It accepts a
transform function that returns a new transformed DataFrame based on its input DF.

Here we have a transform fn that computes average revenue and average units sold then
conditionally adds respective percentage calculations after dollars or units sold as
appropriate.

In [17]:
def my_transform(df_):
    """Adds a 'Result' column to the DataFrame"""
    def my_result(row):
        if row['Metric'] == 'Revenue':
            percentage = (row['Value'] / average_revenue) * 100
            return f"${row['Value']} ({int(percentage)}%)"
        elif row['Metric'] == 'Units Sold':
            percentage = (row['Value'] / average_units) * 100
            return f"{row['Value']} Units ({int(percentage)}%)"
        else:
            return "???"
    average_revenue = df_[df_['Metric'] == 'Revenue']['Value'].mean()
    average_units = df_[df_['Metric'] == 'Units Sold']['Value'].mean()
    df_['Result'] = df_.apply(my_result, axis=1)
    return df_

dt.to_excel(df, sheets=["Store", "Month"], show=True,
            columns=["list", "Product", "Result"],
            transform=my_transform)

#### Combining many parameter

And we can combine these methods to slice the data simultaneously by document and sheet:

In [18]:
dt.to_excel(df, show=True,
            title="Tidy Retail Report",
            folder="/tmp",
            docs=["Month"],
            sheets=["list", "Store"],
            columns=["Product", "Result"],
            transform=my_transform)

# METRICS_MATRIX
The 'metrics_matrix' function is used to create a matrix of metrics from a list of
Dats and a list of metric functions.  The matrix is a 2D array where each row
represents a Dat and each column represents a metric function.  The 'metrics_matrix'
function is useful for creating a matrix of metrics for a set of Dats that can be used
for further analysis or visualization.

The 'metrics_matrix' is designed to be used as a do-method.


In [19]:
show("mr_sprint23")

In [20]:
do("mr_sprint23")

#### Do commands all exported to command line
All do commands can also be run from the command line.  The 'do' launcher script:

In [21]:
!./do mr_sprint23

#### Configuration inheritance
Configuration shadowing inheritance eases testing of special cases.  Here we add a column to the 'mr_sprint23' configuration.

In [22]:
!./do mr_sprint23.add_col

#### Combining configuration and code
Allowing configuration structures to be defined directly in python allows configuration
to be combined with code in the same file.  Combining code and configuration in this
way is powerful, natural, and simple.

In [23]:
#### Cleanup
!rm *.xlsx